<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=cf7ed4e1939b9c4b50c3113b237e588ef1d926b42ed30cfdce42be21f810e446
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 09:10:57
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 1.73 K (0.01%)
Current PnL: -21.62 L (-14.17%)
CY Booked + Current PnL: -8.97 L (-5.88%)
-------------------
Total profit:  2.15 L
Total loss:  -23.76 L
-------------------
Total Booked + Current PnL: 18.01 L (14.34%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.85%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.59 L (61.98%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,63.0,X-LC,7.07,232331.0,21965.0,6807.0,0.0,10.44,2.93,13.68,37.0,3.23,1.64,28.69,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,69.0,M-LC,12.10,175371.0,34531.0,9961.0,0.0,24.52,5.68,31.59,66.0,3.47,1.24,57.50,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,100.85,50.0,M-SC,2.34,86545.0,-14232.0,14323.0,-0.0,-14.12,16.55,0.09,245.0,-0.99,0.61,13.70,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.91,60.0,X-MC,6.67,149880.0,13747.0,16832.0,0.0,10.10,11.23,22.47,91.0,0.82,1.06,24.39,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.16,88515.0,-2163.0,17066.0,0.0,-2.39,19.28,16.44,101.0,-0.13,0.62,17.31,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
1,ABB,7934.00,-42.69,33.0,H-LC,4.97,239616.0,-22003.0,141206.0,0.00,-8.41,58.93,45.57,7.0,-0.16,1.69,1.96,AR,NTT,ELECTRICAL
0,5PAISA,593.00,80.29,18.0,H-SC,16.27,122470.0,-96026.0,124221.0,-0.00,-43.95,101.43,12.90,153.0,-0.77,0.86,0.00,OX40N,NTT,FINANCE
3,ACC,3906.00,-38.07,48.0,X-MC,2.07,184500.0,-53451.0,206105.0,0.00,-22.46,111.71,64.15,174.0,-0.26,1.30,3.49,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-18.12,31.0,H-SC,4.21,76140.0,-24823.0,119357.0,-0.00,-24.59,156.76,93.63,148.0,-0.21,0.54,13.41,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,80.29,18.0,H-SC,16.27,122470.0,-96026.0,124221.0,-0.0,-43.95,101.43,12.90,153.0,-0.77,0.86,0.00,OX40N,NTT,FINANCE
1,ABB,7934.00,-42.69,33.0,H-LC,4.97,239616.0,-22003.0,141206.0,0.0,-8.41,58.93,45.57,7.0,-0.16,1.69,1.96,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.16,88515.0,-2163.0,17066.0,0.0,-2.39,19.28,16.44,101.0,-0.13,0.62,17.31,X40,ATH,PHARMA
3,ACC,3906.00,-38.07,48.0,X-MC,2.07,184500.0,-53451.0,206105.0,0.0,-22.46,111.71,64.15,174.0,-0.26,1.30,3.49,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-18.12,31.0,H-SC,4.21,76140.0,-24823.0,119357.0,-0.0,-24.59,156.76,93.63,148.0,-0.21,0.54,13.41,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,100.85,50.0,M-SC,2.34,86545.0,-14232.0,14323.0,-0.0,-14.12,16.55,0.09,245.0,-0.99,0.61,13.70,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,118.75,74.0,M-SC,5.45,143263.0,-31296.0,31332.0,-0.0,-17.93,21.87,0.02,221.0,-1.00,1.01,24.61,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.31,221238.0,-48429.0,84756.0,0.0,-17.96,38.31,13.47,138.0,-0.57,1.56,12.49,XY24,NTT,PAINTS
17,CERA,9475.0,-25.30,38.0,H-SC,1.82,136240.0,-39663.0,81690.0,-0.0,-22.55,59.96,23.89,149.0,-0.49,0.96,17.66,OX40N,NTT,CERAMICS
67,SIS,528.0,2015.03,46.0,H-SC,2.22,85361.0,-25671.0,49279.0,-0.0,-23.12,57.73,21.26,156.0,-0.52,0.60,15.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,69.0,M-LC,12.1,175371.0,34531.0,9961.0,0.0,24.52,5.68,31.59,66.0,3.47,1.24,57.5,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.18,70.0,M-MC,10.50,248553.0,23591.0,144061.0,-0.0,10.49,57.96,74.53,192.0,0.16,1.75,46.12,XY24,BTT,STEEL
31,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210801.0,5725.0,106855.0,0.0,2.79,50.69,54.89,52.0,0.05,1.49,27.56,X5K,ATH,METALS
79,UNIONBANK,163.00,-2.33,69.0,M-LC,12.10,175371.0,34531.0,9961.0,0.0,24.52,5.68,31.59,66.0,3.47,1.24,57.50,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.94,43.0,M-SC,8.55,118238.0,3440.0,90606.0,-0.0,3.00,76.63,81.92,223.0,0.04,0.83,39.98,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129945.0,6609.0,125020.0,-0.0,5.36,96.21,106.72,119.0,0.05,0.92,29.99,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129945.0,6609.0,125020.0,-0.0,5.36,96.21,106.72,119.0,0.05,0.92,29.99,AR,ATH,MISC
31,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210801.0,5725.0,106855.0,0.0,2.79,50.69,54.89,52.0,0.05,1.49,27.56,X5K,ATH,METALS
85,WHIRLPOOL,2270.00,-42.94,43.0,M-SC,8.55,118238.0,3440.0,90606.0,-0.0,3.00,76.63,81.92,223.0,0.04,0.83,39.98,XR,NTT,DURABLES
86,WIPRO,420.00,-15.15,45.0,M-LC,5.70,148701.0,-2244.0,111704.0,0.0,-1.49,75.12,72.51,53.0,-0.02,1.05,4.45,XR,NTT,IT
46,KPIGREEN,731.05,2.50,43.0,H-SC,9.76,122044.0,-3253.0,62181.0,0.0,-2.60,50.95,47.03,141.0,-0.05,0.86,52.81,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
44,JSWINFRA,342.00,-24.98,24.0,X-MC,4.45,188190.0,-12157.0,42663.0,0.00,-6.07,22.67,15.23,178.0,-0.28,1.33,24.23,X40N,NTT,REALTY
23,DMART,5391.45,-21.15,26.0,X-LC,6.76,200950.0,-9457.0,68624.0,0.00,-4.49,34.15,28.12,38.0,-0.14,1.42,18.07,X40N,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.49,87108.0,-41562.0,86855.0,-0.00,-32.30,99.71,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-14.27,31.0,X-LC,5.98,240880.0,-16183.0,51332.0,-0.00,-6.30,21.31,13.67,24.0,-0.32,1.70,13.72,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.21,39.0,X-MC,0.63,199260.0,-1560.0,69362.0,0.0,-0.78,34.81,33.76,80.0,-0.02,1.40,4.38,X40,ATH,FMCG
41,ITC,452.00,-38.77,44.0,X-LC,0.84,198314.0,-1824.0,22707.0,0.0,-0.91,11.45,10.44,4.0,-0.08,1.40,4.60,X40,NTT,FMCG
66,SIEMENS,4671.50,-4.97,40.0,H-LC,1.59,152720.0,-33375.0,80850.0,0.0,-17.93,52.94,25.51,15.0,-0.41,1.08,12.54,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.30,39.0,X-MC,3.33,306117.0,-21874.0,128416.0,0.0,-6.67,41.95,32.48,92.0,-0.17,2.16,3.99,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.52,58.0,X-LC,3.62,242434.0,8290.0,40341.0,0.0,3.54,16.64,20.77,86.0,0.21,1.71,10.70,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.49,87108.0,-41562.0,86855.0,-0.0,-32.30,99.71,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,34.0,X-SC,37.20,49034.0,-15972.0,166902.0,-0.0,-24.57,340.38,232.18,198.0,-0.10,0.35,0.35,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,39.0,X-MC,7.63,80400.0,-3110.0,23139.0,0.0,-3.72,28.78,23.98,82.0,-0.13,0.57,1.88,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.96,40.0,X-MC,7.12,217120.0,-46245.0,155566.0,0.0,-17.56,71.65,41.51,84.0,-0.30,1.53,1.89,XY25,ATH,FMCG
1,ABB,7934.00,-42.69,33.0,H-LC,4.97,239616.0,-22003.0,141206.0,0.0,-8.41,58.93,45.57,7.0,-0.16,1.69,1.96,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.20,49034.0,-15972.0,166902.0,-0.0,-24.57,340.38,232.18,198.0,-0.10,0.35,0.35,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,39.0,X-MC,7.63,80400.0,-3110.0,23139.0,0.0,-3.72,28.78,23.98,82.0,-0.13,0.57,1.88,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.94,39.0,X-SC,4.04,80455.0,-64705.0,140635.0,0.0,-44.58,174.80,52.31,136.0,-0.46,0.57,7.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.49,87108.0,-41562.0,86855.0,-0.0,-32.30,99.71,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.16,88515.0,-2163.0,17066.0,0.0,-2.39,19.28,16.44,101.0,-0.13,0.62,17.31,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.38,284369.0,-61587.0,128279.0,0.0,-17.80,45.11,19.28,1.0,-0.48,2.00,5.13,X40,ATH,IT
39,INFY,2275.00,-17.09,60.0,X-LC,4.72,322376.0,9488.0,162187.0,0.0,3.03,50.31,54.87,3.0,0.06,2.27,11.57,X40,BTT,IT
41,ITC,452.00,-38.77,44.0,X-LC,0.84,198314.0,-1824.0,22707.0,0.0,-0.91,11.45,10.44,4.0,-0.08,1.40,4.60,X40,NTT,FMCG
83,VBL,671.64,-17.93,49.0,X-LC,5.76,295554.0,-20288.0,132940.0,0.0,-6.42,44.98,35.67,5.0,-0.15,2.08,6.54,X40N,ATH,FMCG
1,ABB,7934.00,-42.69,33.0,H-LC,4.97,239616.0,-22003.0,141206.0,0.0,-8.41,58.93,45.57,7.0,-0.16,1.69,1.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,45.0,L-SC,27.27,78029.0,-35520.0,75571.0,0.0,-31.28,96.85,35.27,268.0,-0.47,0.55,90.98,XR,NTT,HOTELS
7,ASIANTILES,137.00,7561.11,68.0,L-SC,14.07,84082.0,-9728.0,86344.0,0.0,-10.37,102.69,81.67,269.0,-0.11,0.59,62.83,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,48.0,H-SC,7.92,91500.0,-6480.0,28081.0,0.0,-6.61,30.69,22.05,152.0,-0.23,0.64,33.00,XR,ATH,FINANCE
46,KPIGREEN,731.05,2.50,43.0,H-SC,9.76,122044.0,-3253.0,62181.0,0.0,-2.60,50.95,47.03,141.0,-0.05,0.86,52.81,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129945.0,6609.0,125020.0,-0.0,5.36,96.21,106.72,119.0,0.05,0.92,29.99,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,70.80,61.0,H-SC,5.72,147881.0,-34894.0,145959.0,0.0,-19.09,98.70,60.77,125.0,-0.24,1.04,33.18,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7561.11,68.0,L-SC,14.07,84082.0,-9728.0,86344.0,0.0,-10.37,102.69,81.67,269.0,-0.11,0.59,62.83,XR,NTT,CERAMICS
6,ASIANPAINT,3465.66,-7.57,74.0,X-LC,6.17,233235.0,-18533.0,71743.0,-0.0,-7.36,30.76,21.13,27.0,-0.26,1.64,25.46,X40,ATH,PAINTS
5,ANGELONE,3033.00,20.92,65.0,X-SC,6.63,213725.0,22719.0,34987.0,0.0,11.89,16.37,30.21,157.0,0.65,1.51,35.26,X40N,NTT,FINANCE
68,SONACOMS,806.63,-31.67,64.0,M-SC,6.10,84998.0,-16161.0,56167.0,0.0,-15.98,66.08,39.54,202.0,-0.29,0.60,19.85,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,44.54
2,50,75.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.64
LC    33.25
MC    23.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.03
X40      16.41
X40N     12.41
XY25     11.90
XR        9.73
AR        7.77
OX40N     6.59
X200      1.79
X5K       1.49
SR        1.03
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.00
X-LC    22.52
X-MC    18.73
M-SC    12.59
X-SC     5.58
M-LC     5.18
H-LC     4.56
H-MC     2.32
M-MC     1.75
L-SC     1.47
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.19,-4.94,38.87
IT,12.56,-17.98,81.13
FINANCE,12.14,-9.72,59.45
MISC,7.75,-18.14,77.73
PAINTS,5.74,-13.34,30.24
ELECTRICAL,5.69,-12.05,51.77
BANKS,3.97,-19.96,89.75
INSURANCE,3.92,0.49,36.61
AUTO,2.72,-49.09,111.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3314268.0,22
XR,1272905.0,13
AR,1186782.0,9
X40,826093.0,12
X40N,709663.0,9
OX40N,549825.0,9
XY25,485150.0,8
SR,275383.0,2
X5K,106855.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3423901.0,24
M-SC,1260123.0,15
X-MC,1216665.0,14
X-LC,1082303.0,13
X-SC,558742.0,6
H-LC,292218.0,3
M-LC,274503.0,4
H-MC,259215.0,2
L-SC,252205.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188862.0      6
           AR         840238.0      5
           XR         802047.0      7
M-SC       XY24       734131.0      6
X-MC       XY24       575516.0      4
X-LC       X40        444824.0      5
           XY24       310750.0      2
X-SC       X40N       304985.0      4
X-MC       X40        294414.0      6
H-SC       SR         275383.0      2
           OX40N      255190.0      3
X-LC       X40N       241905.0      3
H-LC       AR         222056.0      2
H-MC       XY24       184085.0      1
X-MC       XY25       183962.0      2
X-SC       XY24       166902.0      1
X-MC       X40N       162773.0      2
L-SC       XR         161915.0      2
M-MC       XY24       144061.0      1
M-SC       XR         140203.0      2
           XY25       132086.0      1
           OX40N      129215.0      4
           AR         124488.0      2
M-LC       XR         111704.0      1
           X5K        106855.0      1
L-SC       OX40N       90290.0      1
X-SC       X40         86855.0      1
X-LC       XY25        84824.0      3
H-MC       OX40N       75130.0      1
H-LC       X200        70162.0      1
H-SC       MH          62181.0      1
L-MC       XR          57036.0      1
M-LC       XY25        45983.0      1
L-LC       XY25        38295.0      1
M-LC       XY24         9961.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
